In [2]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
import matplotlib.pyplot as plt

1.3.0


In [1]:
tf.reset_default_graph()

NameError: name 'tf' is not defined

In [3]:
iterator, filename = get_train_inputs(batch_size=100,repeat=False,num_classes=2)

In [ ]:
i,l = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
    next_element = iterator.get_next()
    while True:
        image, label = sess.run(next_element)
        plt.imshow(image)
        plt.show()
    #plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)

In [ ]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

# train

In [4]:
encoder = SlimModelEncoder(name="vgg_16",num_classes=2,is_training=True)

In [5]:
image, label = iterator.get_next()
init_fn, end_points = encoder.build(image)

INFO:tensorflow:Scale of 0 disables regularizer.


In [6]:
end_points.keys()

odict_keys(['vgg_16/conv1/conv1_1', 'vgg_16/conv1/conv1_2', 'vgg_16/pool1', 'vgg_16/conv2/conv2_1', 'vgg_16/conv2/conv2_2', 'vgg_16/pool2', 'vgg_16/conv3/conv3_1', 'vgg_16/conv3/conv3_2', 'vgg_16/conv3/conv3_3', 'vgg_16/pool3', 'vgg_16/conv4/conv4_1', 'vgg_16/conv4/conv4_2', 'vgg_16/conv4/conv4_3', 'vgg_16/pool4', 'vgg_16/conv5/conv5_1', 'vgg_16/conv5/conv5_2', 'vgg_16/conv5/conv5_3', 'vgg_16/pool5', 'vgg_16/fc6', 'vgg_16/fc7', 'vgg_16/fc8'])

In [7]:
end_points['vgg_16/conv5/conv5_3'], end_points['vgg_16/conv4/conv4_3'], end_points['vgg_16/conv3/conv3_3']

(<tf.Tensor 'vgg_16/conv5/conv5_3/Relu:0' shape=(1, 14, 14, 512) dtype=float32>,
 <tf.Tensor 'vgg_16/conv4/conv4_3/Relu:0' shape=(1, 28, 28, 512) dtype=float32>,
 <tf.Tensor 'vgg_16/conv3/conv3_3/Relu:0' shape=(1, 56, 56, 256) dtype=float32>)

In [6]:
decoder = FCNDecoder(end_points, nb_classes=21, scope='fcn')

In [7]:
tensors_to_connect = OrderedDict()
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (2,2)
tensors_to_connect['vgg_16/conv3/conv3_3'] = (4,4)

In [8]:
net = decoder.build(tensors_to_connect)

In [9]:
net.get_shape()

TensorShape([Dimension(1), Dimension(224), Dimension(224), Dimension(21)])

In [10]:
tf.get_default_graph().get_operations()

[<tf.Operation 'training_data/Placeholder' type=Placeholder>,
 <tf.Operation 'training_data/compression_type' type=Const>,
 <tf.Operation 'training_data/Iterator' type=Iterator>,
 <tf.Operation 'training_data/TFRecordDataset' type=TFRecordDataset>,
 <tf.Operation 'training_data/MapDataset' type=MapDataset>,
 <tf.Operation 'training_data/MakeIterator' type=MakeIterator>,
 <tf.Operation 'IteratorGetNext' type=IteratorGetNext>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'Shape' type=Shape>,
 <tf.Operation 'strided_slice/stack' type=Const>,
 <tf.Operation 'strided_slice/stack_1' type=Const>,
 <tf.Operation 'strided_slice/stack_2' type=Const>,
 <tf.Operation 'strided_slice' type=StridedSlice>,
 <tf.Operation 'strided_slice_1/stack' type=Const>,
 <tf.Operation 'strided_slice_1/stack_1' type=Const>,
 <tf.Operation 'strided_slice_1/stack_2' type=Const>,
 <tf.Operation 'strided_slice_1' type=StridedSlice>,
 <tf.Operation 'ToFloat' type=Cast>,
 <tf.Operation 'ToFloat_1' type=Cast>,
 <tf.